In [274]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
%matplotlib inline


In [275]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')
home_directory = os.path.expanduser("~")
omnetpp_directory = os.path.join(home_directory, "omnetpp-6.0.2")
vec_directory = os.path.join(omnetpp_directory, "samples", "thesis", "results")

In [276]:
dispatchingAlgos = ["Random", "Greedy", "ModifiedGreedy", "Proposed"] # ["Random", "Greedy", "Proposed"]
schedulingAlgos = ["FIFO", "FIFO", "FIFO", "Proposed"] # ["Random", "Greedy", "Proposed"]
numUsers = [user for user in range(1000, 1001, 200)]
repetition = 1
for idx, dispatchingAlgo in enumerate(dispatchingAlgos):
    for numUser in numUsers:
        for i in range(repetition):
            command_vec = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/General-numUsers={numUser},dispatchingAlgo={dispatchingAlgo},schedulingAlgo={schedulingAlgos[idx]}-#{i}-vec.csv {vec_directory}/General-numUsers={numUser},dispatchingAlgo={dispatchingAlgo},schedulingAlgo={schedulingAlgos[idx]}-#{i}.vec"
            command_sca = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/General-numUsers={numUser},dispatchingAlgo={dispatchingAlgo},schedulingAlgo={schedulingAlgos[idx]}-#{i}-sca.csv {vec_directory}/General-numUsers={numUser},dispatchingAlgo={dispatchingAlgo},schedulingAlgo={schedulingAlgos[idx]}-#{i}.sca"
            os.system(command_vec)
            os.system(command_sca)


Exported 125 vectors
Exported 6 scalars, 11855 parameters
Exported 125 vectors
Exported 6 scalars, 11855 parameters
Exported 125 vectors
Exported 6 scalars, 11855 parameters
Exported 125 vectors
Exported 6 scalars, 11855 parameters


In [277]:

algoCompletionRate = []
algoCompletionTime = []

for idx, dispatchingAlgo in enumerate(dispatchingAlgos):
    completionRateWithDiffNumUsers = []
    completionTimeWithDiffNumUsers = []
    for numUser in numUsers:
        accumulateCompletionRate = 0
        accumulateCompletionTime = 0
        for i in range(repetition):
            sca_out = pd.read_csv(f"General-numUsers={numUser},dispatchingAlgo={dispatchingAlgo},schedulingAlgo={schedulingAlgos[idx]}-#{i}-sca.csv")
            print(f"{dispatchingAlgo}-{numUser}-{i}")
            display(sca_out[(sca_out['type'] == 'scalar')])
            # sca_mean_row = sca_out[(sca_out['type'] == 'scalar') & (sca_out['name'] == 'isCompleted:mean')]
            sca_mean_row = sca_out[(sca_out['type'] == 'scalar')]
            accumulateCompletionRate += float(sca_mean_row[sca_mean_row['name'] == 'isCompleted:mean'].iloc[0]['value'])
            accumulateCompletionTime += float(sca_mean_row[sca_mean_row['name'] == 'totalTime:mean'].iloc[0]['value'])
        avgCompletionRate = accumulateCompletionRate / repetition
        avgCompletionTime = accumulateCompletionTime / repetition
        completionRateWithDiffNumUsers.append(avgCompletionRate)
        completionTimeWithDiffNumUsers.append(avgCompletionTime)
    algoCompletionRate.append(completionRateWithDiffNumUsers)
    algoCompletionTime.append(completionTimeWithDiffNumUsers)

for idx, dispatchingAlgo in enumerate(dispatchingAlgos):
    print(f"{dispatchingAlgo} completionRate is {algoCompletionRate[idx]}")
    print(f"{dispatchingAlgo} completionTime is {algoCompletionTime[idx]}")


Random-1000-0


,run,type,module,name,attrname,attrvalue,value
43,General-0-20240220-03:35:43-39264,scalar,EdgeEnv.finishedTaskCollector,totalTime:mean,NaN,NaN,0.027170758508806
47,General-0-20240220-03:35:43-39264,scalar,EdgeEnv.finishedTaskCollector,partialComplete:count,NaN,NaN,0
51,General-0-20240220-03:35:43-39264,scalar,EdgeEnv.finishedTaskCollector,partialComplete:sum,NaN,NaN,0
55,General-0-20240220-03:35:43-39264,scalar,EdgeEnv.finishedTaskCollector,isCompleted:count,NaN,NaN,598294
59,General-0-20240220-03:35:43-39264,scalar,EdgeEnv.finishedTaskCollector,isCompleted:sum,NaN,NaN,596557
63,General-0-20240220-03:35:43-39264,scalar,EdgeEnv.finishedTaskCollector,isCompleted:mean,NaN,NaN,0.99709674507851


Greedy-1000-0


,run,type,module,name,attrname,attrvalue,value
43,General-1-20240220-03:35:43-39263,scalar,EdgeEnv.finishedTaskCollector,totalTime:mean,NaN,NaN,0.026238784431519
47,General-1-20240220-03:35:43-39263,scalar,EdgeEnv.finishedTaskCollector,partialComplete:count,NaN,NaN,0
51,General-1-20240220-03:35:43-39263,scalar,EdgeEnv.finishedTaskCollector,partialComplete:sum,NaN,NaN,0
55,General-1-20240220-03:35:43-39263,scalar,EdgeEnv.finishedTaskCollector,isCompleted:count,NaN,NaN,598695
59,General-1-20240220-03:35:43-39263,scalar,EdgeEnv.finishedTaskCollector,isCompleted:sum,NaN,NaN,598664
63,General-1-20240220-03:35:43-39263,scalar,EdgeEnv.finishedTaskCollector,isCompleted:mean,NaN,NaN,0.99994822071338


ModifiedGreedy-1000-0


,run,type,module,name,attrname,attrvalue,value
43,General-2-20240220-03:35:45-39277,scalar,EdgeEnv.finishedTaskCollector,totalTime:mean,NaN,NaN,0.026589451818044
47,General-2-20240220-03:35:45-39277,scalar,EdgeEnv.finishedTaskCollector,partialComplete:count,NaN,NaN,0
51,General-2-20240220-03:35:45-39277,scalar,EdgeEnv.finishedTaskCollector,partialComplete:sum,NaN,NaN,0
55,General-2-20240220-03:35:45-39277,scalar,EdgeEnv.finishedTaskCollector,isCompleted:count,NaN,NaN,598696
59,General-2-20240220-03:35:45-39277,scalar,EdgeEnv.finishedTaskCollector,isCompleted:sum,NaN,NaN,598678
63,General-2-20240220-03:35:45-39277,scalar,EdgeEnv.finishedTaskCollector,isCompleted:mean,NaN,NaN,0.99996993465799


Proposed-1000-0


,run,type,module,name,attrname,attrvalue,value
43,General-3-20240220-03:35:45-39282,scalar,EdgeEnv.finishedTaskCollector,totalTime:mean,NaN,NaN,0.021390849494185
47,General-3-20240220-03:35:45-39282,scalar,EdgeEnv.finishedTaskCollector,partialComplete:count,NaN,NaN,176
51,General-3-20240220-03:35:45-39282,scalar,EdgeEnv.finishedTaskCollector,partialComplete:sum,NaN,NaN,17
55,General-3-20240220-03:35:45-39282,scalar,EdgeEnv.finishedTaskCollector,isCompleted:count,NaN,NaN,598696
59,General-3-20240220-03:35:45-39282,scalar,EdgeEnv.finishedTaskCollector,isCompleted:sum,NaN,NaN,498702
63,General-3-20240220-03:35:45-39282,scalar,EdgeEnv.finishedTaskCollector,isCompleted:mean,NaN,NaN,0.83298034394751


Random completionRate is [0.99709674507851]
Random completionTime is [0.027170758508806]
Greedy completionRate is [0.99994822071338]
Greedy completionTime is [0.026238784431519]
ModifiedGreedy completionRate is [0.99996993465799]
ModifiedGreedy completionTime is [0.026589451818044]
Proposed completionRate is [0.83298034394751]
Proposed completionTime is [0.021390849494185]


In [278]:
proposed_500_vec_out = pd.read_csv(f"General-numUsers=500,dispatchingAlgo=Proposed,schedulingAlgo=Proposed-#0-vec.csv")
proposed_500_vec_out = proposed_500_vec_out[proposed_500_vec_out['type'] == 'vector']
tmp = proposed_500_vec_out['vecvalue'].str.split(expand=True)
tmp = tmp.transpose()
proposed_500_vec_out['name']
# print([name for name in proposed_500_vec_out['name'].to_list()])


FileNotFoundError: [Errno 2] No such file or directory: 'General-numUsers=500,dispatchingAlgo=Proposed,schedulingAlgo=Proposed-#0-vec.csv'

In [ ]:
display(tmp)

,40,44,48,52,56,60,64,68,72,76,...,500,504,508,512,516,520,524,528,532,536
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.18,-0.09,896,99,797,-0.04,-0.02,82,69,13,...,0,0,1,1,0,0,0,1,1,0
2,-0.18,-0.09,998,100,898,-0.06,-0.03,13,13,0,...,0,0,1,1,0,0,0,1,1,0
3,-0.02,-0.01,1064,100,964,-0.016,-0.008,13,13,0,...,0.02,0.01,0,0,0,0,0,2,2,0
4,-0.68,-0.34,1031,100,931,-0.06,-0.03,9,9,0,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,-0.86,-0.43,964,100,864,-0.06,-0.03,3,3,0,...,0,0,3,3,0,0,0,1,1,0
297,-0.8,-0.4,936,100,836,-0.06,-0.03,3,3,0,...,0,0,4,4,0,0,0,2,2,0
298,-0.68,-0.34,1074,100,974,-0.06,-0.03,2,2,0,...,0,0,4,4,0,0,0,1,1,0
299,-0.5,-0.25,1023,100,923,-0.06,-0.03,0,0,0,...,0,0,4,4,0,0,0,0,0,0


In [ ]:
# random_command_vec = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/random_vec.csv {vec_directory}/Random-#0.vec"
# random_command_sca = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/random_sca.csv {vec_directory}/Random-#0.sca"

# os.system(random_command_vec)
# os.system(random_command_sca)
# random_out = pd.read_csv('random_vec.csv')
# display(random_out.head())
# print(random_out['type'].unique())

In [ ]:
# random_vector_type_df = random_out[random_out['type'] == 'vector']
# display(random_vector_type_df)
# random_vecval_df = random_vector_type_df['vecvalue'].str.split(expand=True)
# display(random_vecval_df)
# random_tasks_df = random_vecval_df.transpose()
# random_tasks_df.columns = [name.split(':')[0] for name in random_vector_type_df['name'].to_list()]
# # display(random_tasks_df)
# random_tasks_numeric_df = random_tasks_df.apply(pd.to_numeric, errors='ignore')
# display(random_tasks_numeric_df)

In [ ]:
# greedy_command_vec = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/greedy_vec.csv {vec_directory}/Greedy-#0.vec"
# greedy_command_sca = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/greedy_sca.csv {vec_directory}/Greedy-#0.sca"

# os.system(greedy_command_vec)
# os.system(greedy_command_sca)

# greedy_out = pd.read_csv('greedy_vec.csv')
# display(greedy_out.head())

In [ ]:
# greedy_vector_type_df = greedy_out[greedy_out['type'] == 'vector']
# display(greedy_vector_type_df)
# greedy_vecval_df = greedy_vector_type_df['vecvalue'].str.split(expand=True)
# display(greedy_vecval_df)
# greedy_tasks_df = greedy_vecval_df.transpose()
# greedy_tasks_df.columns = [name.split(':')[0] for name in greedy_vector_type_df['name'].to_list()]
# # display(greedy_tasks_df)
# greedy_tasks_numeric_df = greedy_tasks_df.apply(pd.to_numeric, errors='ignore')
# display(greedy_tasks_numeric_df)


In [ ]:
# proposed_command_vec = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/proposed_vec.csv {vec_directory}/Proposed-#0.vec"
# proposed_command_sca = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/proposed_sca.csv {vec_directory}/Proposed-#0.sca"

# os.system(proposed_command_vec)
# os.system(proposed_command_sca)

# proposed_out = pd.read_csv('proposed_vec.csv')
# display(proposed_out.head())

In [ ]:
# proposed_vector_type_df = proposed_out[proposed_out['type'] == 'vector']
# display(proposed_vector_type_df)
# proposed_vecvalue_column_df = proposed_vector_type_df[['vecvalue']]

In [ ]:
# proposed_vecval_df = proposed_vecvalue_column_df['vecvalue'].str.split(expand=True)
# display(proposed_vecval_df)

In [ ]:
# proposed_tasks_df = proposed_vecval_df.transpose()
# proposed_tasks_df.columns = [name.split(':')[0] for name in proposed_vector_type_df['name'].to_list()]
# # display(proposed_tasks_df)
# proposed_tasks_numeric_df = proposed_tasks_df.apply(pd.to_numeric, errors='ignore')
# display(proposed_tasks_numeric_df)

In [ ]:
# print(random_tasks_numeric_df[random_tasks_numeric_df['isCompleted'] == 1].size / random_tasks_numeric_df.size)
# print(greedy_tasks_numeric_df[greedy_tasks_numeric_df['isCompleted'] == 1].size / greedy_tasks_numeric_df.size)
# print(proposed_tasks_numeric_df[proposed_tasks_numeric_df['isCompleted'] == 1].size / proposed_tasks_numeric_df.size)